In [7]:
from transformers import CONFIG_MAPPING, MODEL_MAPPING
from transformers.models.auto.configuration_auto import _LazyConfigMapping
from transformers.models.auto.auto_factory import _LazyAutoMapping
from transformers.models.auto import MODEL_FOR_CAUSAL_LM_MAPPING

from models.flash_stu_550M.model import FlashSTU, FlashSTUConfig
import lm_eval as evaluator
import os

os.environ["HF_ALLOW_CODE_EVAL"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["HF_HUB_OFFLINE"] = "1"

# Dynamically register FlashSTU:
if isinstance(CONFIG_MAPPING, _LazyConfigMapping):
    CONFIG_MAPPING._extra_content["flashstu"] = FlashSTUConfig
else:
    CONFIG_MAPPING.update({"flashstu": FlashSTUConfig})

if isinstance(MODEL_MAPPING, _LazyAutoMapping):
    MODEL_MAPPING._extra_content[FlashSTUConfig] = FlashSTU
else:
    MODEL_MAPPING.update({FlashSTUConfig: FlashSTU})

if isinstance(MODEL_FOR_CAUSAL_LM_MAPPING, _LazyAutoMapping):
    MODEL_FOR_CAUSAL_LM_MAPPING._extra_content[FlashSTUConfig] = FlashSTU
else:
    MODEL_FOR_CAUSAL_LM_MAPPING.update({FlashSTUConfig: FlashSTU})

print("Registered FlashSTU model and configuration.")

# Define evaluation tasks.
tasks = [
    "mmlu",
    "hellaswag",
    "piqa",
    "boolq",
    "winogrande",
    "commonsense_qa",
    "openbookqa",
    "arc_easy",
    "arc_challenge"
]

# Few-shot settings dictionary.
tasks_fewshot = {
    "hellaswag": 0,
    "mmlu": 5,
    "piqa": 0,
    "boolq": 0,
    "winogrande": -1,
    "commonsense_qa": 7,
    "openbookqa": -1,
    "arc_easy": -1,
    "arc_challenge": -1,
}

all_results = {}

for task in tasks:
    print(f"Evaluating task: {task}")
    eval_kwargs = dict(
        model="hf",
        model_args=(
            "pretrained=Hazan-Lab/Flash_STU_550M,"
            "trust_remote_code=True,"
            "dtype=bfloat16,"
            "cache_dir=/path/to/your/cache"
        ),
        tasks=[task],
        batch_size=1,
        device="cuda:0"
    )
    few_shot_value = tasks_fewshot.get(task, -1)
    if few_shot_value != -1:
        eval_kwargs["num_fewshot"] = few_shot_value

    results = evaluator.simple_evaluate(**eval_kwargs)
    task_result = results["results"].get(task, {})
    all_results[task] = task_result
    print(f"Results for {task}:")
    print(task_result)
    print("\n" + "="*50 + "\n")

print("All Evaluation Results:")
for task, result in all_results.items():
    print(f"{task}: {result}")



Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Registered FlashSTU model and configuration.
Evaluating task: mmlu


OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like Hazan-Lab/Flash_STU_550M is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.